<a href="https://colab.research.google.com/github/almamyr/LakeAtitlan/blob/main/Lake_Atitlan_Discharge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install GEOGloWS package
!pip install geoglows
import geoglows
from IPython.core.display import display, HTML
from google.colab import files

In [ ]:
# Import other packages
from scipy import integrate
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

In [ ]:
# Go collect the data for each river that flows into the lake
reach_id_a = 951870
stats_a = geoglows.streamflow.forecast_stats(reach_id_a)

reach_id_b = 951853
stats_b = geoglows.streamflow.forecast_stats(reach_id_b)

reach_id_c = 951857
stats_c = geoglows.streamflow.forecast_stats(reach_id_c)

In [ ]:
# Eliminate blank rows in the dataframe
stats_a = stats_a[stats_a["flow_min_m^3/s"].notna()]
stats_b = stats_b[stats_b["flow_min_m^3/s"].notna()]
stats_c = stats_c[stats_c["flow_min_m^3/s"].notna()]

In [ ]:
# Convert to datetime
stats_a.index = pd.to_datetime(stats_a.index)
stats_b.index = pd.to_datetime(stats_b.index)
stats_c.index = pd.to_datetime(stats_c.index)

In [ ]:
# Get first timestep of each day
midnight_a = stats_a[stats_a.index.hour == 0]
midnight_b = stats_b[stats_b.index.hour == 0]
midnight_c = stats_c[stats_c.index.hour == 0]

In [ ]:
# Create a dataset for each day of each dataframe

for i in range (1,16):
  locals()["day"+str(i)+"_a"] = stats_a[stats_a.index.dayofyear == pd.to_datetime('today').dayofyear+(i-2)]
  locals()["day"+str(i)+"_b"] = stats_b[stats_b.index.dayofyear == pd.to_datetime('today').dayofyear+(i-2)]
  locals()["day"+str(i)+"_c"] = stats_c[stats_c.index.dayofyear == pd.to_datetime('today').dayofyear+(i-2)]

  # Add the first timestep of the following day to the previous day's dataset
  locals()["day"+str(i)+"_a"] = locals()["day"+str(i)+"_a"].append(midnight_a[midnight_a.index.dayofyear == pd.to_datetime('today').dayofyear+(i-1)])
  locals()["day"+str(i)+"_b"] = locals()["day"+str(i)+"_b"].append(midnight_b[midnight_b.index.dayofyear == pd.to_datetime('today').dayofyear+(i-1)])
  locals()["day"+str(i)+"_c"] = locals()["day"+str(i)+"_c"].append(midnight_c[midnight_c.index.dayofyear == pd.to_datetime('today').dayofyear+(i-1)])



## Integrate Data

### Minimum 

In [ ]:
# Integrate minimum volumes over each day
# Timestep for the first 5 days is 3 hours (10800s), after that 6 hours (21600s)

for i in range (1,16):
  if i<6:
    locals()["min_day"+str(i)+"_a"] = integrate.trapz(y = locals()["day"+str(i)+"_a"]["flow_min_m^3/s"], dx = 10800)
    locals()["min_day"+str(i)+"_b"] = integrate.trapz(y = locals()["day"+str(i)+"_b"]["flow_min_m^3/s"], dx = 10800)
    locals()["min_day"+str(i)+"_c"] = integrate.trapz(y = locals()["day"+str(i)+"_c"]["flow_min_m^3/s"], dx = 10800)
  if i>=6:
    locals()["min_day"+str(i)+"_a"] = integrate.trapz(y = locals()["day"+str(i)+"_a"]["flow_min_m^3/s"], dx = 21600)
    locals()["min_day"+str(i)+"_b"] = integrate.trapz(y = locals()["day"+str(i)+"_b"]["flow_min_m^3/s"], dx = 21600)
    locals()["min_day"+str(i)+"_c"] = integrate.trapz(y = locals()["day"+str(i)+"_c"]["flow_min_m^3/s"], dx = 21600)

In [ ]:
# Create an array for each stream containing daily minimum values

min_a_array = np.zeros(15)
min_b_array = np.zeros(15)
min_c_array = np.zeros(15)

for i in range (1,16): 
  min_a_array[i-1] = locals()["min_day"+str(i)+"_a"]
  min_b_array[i-1] = locals()["min_day"+str(i)+"_b"]
  min_c_array[i-1] = locals()["min_day"+str(i)+"_c"]

In [ ]:
# Add the min arrays to get total daily minimum volume
min_array = min_a_array + min_b_array + min_c_array

### Maximum

In [ ]:
# Integrate maximum values over each day
# Timestep for the first 5 days is 3 hours (10800s), after that 6 hours (21600s)

for i in range (1,16):
  if i<6:
    locals()["max_day"+str(i)+"_a"] = integrate.trapz(y = locals()["day"+str(i)+"_a"]["flow_max_m^3/s"], dx = 10800)
    locals()["max_day"+str(i)+"_b"] = integrate.trapz(y = locals()["day"+str(i)+"_b"]["flow_max_m^3/s"], dx = 10800)
    locals()["max_day"+str(i)+"_c"] = integrate.trapz(y = locals()["day"+str(i)+"_c"]["flow_max_m^3/s"], dx = 10800)
  if i>=6:
    locals()["max_day"+str(i)+"_a"] = integrate.trapz(y = locals()["day"+str(i)+"_a"]["flow_max_m^3/s"], dx = 21600)
    locals()["max_day"+str(i)+"_b"] = integrate.trapz(y = locals()["day"+str(i)+"_b"]["flow_max_m^3/s"], dx = 21600)
    locals()["max_day"+str(i)+"_c"] = integrate.trapz(y = locals()["day"+str(i)+"_c"]["flow_max_m^3/s"], dx = 21600)

In [ ]:
# Create an array for each stream containing daily maximum values

max_a_array = np.zeros(15)
max_b_array = np.zeros(15)
max_c_array = np.zeros(15)

for i in range (1,16): 
  max_a_array[i-1] = locals()["max_day"+str(i)+"_a"]
  max_b_array[i-1] = locals()["max_day"+str(i)+"_b"]
  max_c_array[i-1] = locals()["max_day"+str(i)+"_c"]

In [ ]:
# Add the max arrays to get total daily maximum volume
max_array = max_a_array + max_b_array + max_c_array

### Average

In [ ]:
# Integrate average values over each day
# Timestep for the first 5 days is 3 hours (10800s), after that 6 hours (21600s)

for i in range (1,16):
  if i<6:
    locals()["avg_day"+str(i)+"_a"] = integrate.trapz(y = locals()["day"+str(i)+"_a"]["flow_avg_m^3/s"], dx = 10800)
    locals()["avg_day"+str(i)+"_b"] = integrate.trapz(y = locals()["day"+str(i)+"_b"]["flow_avg_m^3/s"], dx = 10800)
    locals()["avg_day"+str(i)+"_c"] = integrate.trapz(y = locals()["day"+str(i)+"_c"]["flow_avg_m^3/s"], dx = 10800)
  if i>=6:
    locals()["avg_day"+str(i)+"_a"] = integrate.trapz(y = locals()["day"+str(i)+"_a"]["flow_avg_m^3/s"], dx = 21600)
    locals()["avg_day"+str(i)+"_b"] = integrate.trapz(y = locals()["day"+str(i)+"_b"]["flow_avg_m^3/s"], dx = 21600)
    locals()["avg_day"+str(i)+"_c"] = integrate.trapz(y = locals()["day"+str(i)+"_c"]["flow_avg_m^3/s"], dx = 21600)

In [ ]:
# Create an array for each stream containing daily mean values

avg_a_array = np.zeros(15)
avg_b_array = np.zeros(15)
avg_c_array = np.zeros(15)

for i in range (1,16): 
  avg_a_array[i-1] = locals()["avg_day"+str(i)+"_a"]
  avg_b_array[i-1] = locals()["avg_day"+str(i)+"_b"]
  avg_c_array[i-1] = locals()["avg_day"+str(i)+"_c"]

In [ ]:
# Add the avg arrays to get total daily average volume
avg_array = avg_a_array + avg_b_array + avg_c_array

## Output

In [ ]:
print("Minimum:", min_array)
print("Maximum:", max_array)
print("Average:", avg_array)